# Tutorial

Using ibis + duckdb to do large scale (spatial) data analysis

In [14]:
import ibis as ib
from ibis import _

ib.options.interactive = True

## Introduction

ibis is the link between powerfull backend infrastructure and user friendly `python`. It allows writing code in a familiar environement and with an almost same syntax as `pandas`, but executes the code in a database

In [32]:
conn = ib.connect("duckdb://", memory_limit="50GB", threads=6)

In [33]:
# type(conn)

In [34]:
conn.list_catalogs()

['memory', 'system', 'temp']

In [35]:
conn.list_databases()

['information_schema', 'main', 'pg_catalog']

In [36]:
conn.list_tables()

[]

In [37]:
conn.sql("SELECT * FROM duckdb_settings();").execute()

,name,value,description,input_type,scope
0,access_mode,automatic,"Access mode of the database (AUTOMATIC, READ_O...",VARCHAR,GLOBAL
1,allocator_background_threads,false,Whether to enable the allocator background thr...,BOOLEAN,GLOBAL
2,allocator_bulk_deallocation_flush_threshold,512.0 MiB,If a bulk deallocation larger than this occurs...,VARCHAR,GLOBAL
3,allocator_flush_threshold,128.0 MiB,Peak allocation threshold at which to flush th...,VARCHAR,GLOBAL
4,allow_community_extensions,true,Allow to load community built extensions,BOOLEAN,GLOBAL
...,...,...,...,...,...
124,disable_parquet_prefetching,false,Disable the prefetching mechanism in Parquet,BOOLEAN,GLOBAL
125,python_scan_all_frames,false,"If set, restores the old behavior of scanning ...",BOOLEAN,GLOBAL
126,python_enable_replacements,false,Whether variables visible to the current stack...,BOOLEAN,LOCAL
127,binary_as_string,,"In Parquet files, interpret binary data as a s...",BOOLEAN,GLOBAL


## Reading some data

In [40]:
iris = ib.examples.iris_raw.fetch(table_name="iris",backend=conn)

In [41]:
conn.list_tables()

['iris', 'iris_raw']

Names on different sides of the process can be a source of confucion !!

#### Schema

In [42]:
iris.schema()

ibis.Schema {
  Sepal.Length  float64
  Sepal.Width   float64
  Petal.Length  float64
  Petal.Width   float64
  Species       string
}

#### Head/tail

In [43]:
iris.head()

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Sepal.Length ┃ Sepal.Width ┃ Petal.Length ┃ Petal.Width ┃ Species ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━┩
│ float64      │ float64     │ float64      │ float64     │ string  │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────┤
│          5.1 │         3.5 │          1.4 │         0.2 │ setosa  │
│          4.9 │         3.0 │          1.4 │         0.2 │ setosa  │
│          4.7 │         3.2 │          1.3 │         0.2 │ setosa  │
│          4.6 │         3.1 │          1.5 │         0.2 │ setosa  │
│          5.0 │         3.6 │          1.4 │         0.2 │ setosa  │
└──────────────┴─────────────┴──────────────┴─────────────┴─────────┘

In [48]:
iris.sample(.1)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sepal.Length ┃ Sepal.Width ┃ Petal.Length ┃ Petal.Width ┃ Species    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64      │ float64     │ float64      │ float64     │ string     │
├──────────────┼─────────────┼──────────────┼─────────────┼────────────┤
│          5.0 │         3.6 │          1.4 │         0.2 │ setosa     │
│          4.8 │         3.4 │          1.6 │         0.2 │ setosa     │
│          5.7 │         4.4 │          1.5 │         0.4 │ setosa     │
│          4.6 │         3.6 │          1.0 │         0.2 │ setosa     │
│          5.2 │         3.4 │          1.4 │         0.2 │ setosa     │
│          5.3 │         3.7 │          1.5 │         0.2 │ setosa     │
│          5.9 │         3.0 │          4.2 │         1.5 │ versicolor │
│          5.8 │         2.7 │          3.9 │         1.2 │ versicolor │
│          6.3 │         2.3 │          4.4 │         1.3 │ versicolor │
│          5.7 │         3.0 │          4.2 │         1.2 │ versicolor │
│            … │           … │            … │           … │ …          │
└──────────────┴─────────────┴──────────────┴─────────────┴────────────┘

#### Describe

In [51]:
iris.describe()

┏━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ name         ┃ pos   ┃ type    ┃ count ┃ nulls ┃ unique ┃ mode   ┃ mean     ┃ std      ┃ min     ┃ p25     ┃ p50     ┃ p75     ┃ max     ┃
┡━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ string       │ int16 │ string  │ int64 │ int64 │ int64  │ string │ float64  │ float64  │ float64 │ float64 │ float64 │ float64 │ float64 │
├──────────────┼───────┼─────────┼───────┼───────┼────────┼────────┼──────────┼──────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ Petal.Length │     2 │ float64 │   150 │     0 │     43 │ NULL   │ 3.758000 │ 1.765298 │     1.0 │     1.6 │    4.35 │     5.1 │     6.9 │
│ Petal.Width  │     3 │ float64 │   150 │     0 │     22 │ NULL   │ 1.199333 │ 0.762238 │     0.1 │     0.3 │    1.30 │     1.8 │     2.5 │
│ Species      │     4 │ string  │   150 │     0 │      3 │ setosa │     NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ Sepal.Length │     0 │ float64 │   150 │     0 │     35 │ NULL   │ 5.843333 │ 0.828066 │     4.3 │     5.1 │    5.80 │     6.4 │     7.9 │
│ Sepal.Width  │     1 │ float64 │   150 │     0 │     23 │ NULL   │ 3.057333 │ 0.435866 │     2.0 │     2.8 │    3.00 │     3.3 │     4.4 │
└──────────────┴───────┴─────────┴───────┴───────┴────────┴────────┴──────────┴──────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

In [ ]:
ib.to_sql(iris.describe())

```sql
SELECT
  *
FROM (
  SELECT
    *
  FROM (
    SELECT
      *
    FROM (
      SELECT
        'Petal.Length' AS "name",
        2 AS "pos",
        'float64' AS "type",
        COUNT("t0"."Petal.Length" IS NULL) AS "count",
        SUM(CAST("t0"."Petal.Length" IS NULL AS INT)) AS "nulls",
        COUNT(DISTINCT "t0"."Petal.Length") AS "unique",
        CAST(NULL AS TEXT) AS "mode",
        AVG("t0"."Petal.Length") AS "mean",
        STDDEV_SAMP("t0"."Petal.Length") AS "std",
        MIN("t0"."Petal.Length") AS "min",
        QUANTILE_CONT("t0"."Petal.Length", 0.25) AS "p25",
        QUANTILE_CONT("t0"."Petal.Length", 0.5) AS "p50",
        QUANTILE_CONT("t0"."Petal.Length", 0.75) AS "p75",
        MAX("t0"."Petal.Length") AS "max"
      FROM "iris" AS "t0"
    ) AS "t1"
  ) AS "t6"
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      *
    FROM (
      SELECT
        'Petal.Width' AS "name",
        3 AS "pos",
        'float64' AS "type",
        COUNT("t0"."Petal.Width" IS NULL) AS "count",
        SUM(CAST("t0"."Petal.Width" IS NULL AS INT)) AS "nulls",
        COUNT(DISTINCT "t0"."Petal.Width") AS "unique",
        CAST(NULL AS TEXT) AS "mode",
        AVG("t0"."Petal.Width") AS "mean",
        STDDEV_SAMP("t0"."Petal.Width") AS "std",
        MIN("t0"."Petal.Width") AS "min",
        QUANTILE_CONT("t0"."Petal.Width", 0.25) AS "p25",
        QUANTILE_CONT("t0"."Petal.Width", 0.5) AS "p50",
        QUANTILE_CONT("t0"."Petal.Width", 0.75) AS "p75",
        MAX("t0"."Petal.Width") AS "max"
      FROM "iris" AS "t0"
    ) AS "t2"
  ) AS "t7"
) AS "t11"
UNION ALL
SELECT
  *
FROM (
  SELECT
    *
  FROM (
    SELECT
      *
    FROM (
      SELECT
        'Species' AS "name",
        4 AS "pos",
        'string' AS "type",
        COUNT("t0"."Species" IS NULL) AS "count",
        SUM(CAST("t0"."Species" IS NULL AS INT)) AS "nulls",
        COUNT(DISTINCT "t0"."Species") AS "unique",
        MODE("t0"."Species") AS "mode",
        CAST(NULL AS DOUBLE) AS "mean",
        CAST(NULL AS DOUBLE) AS "std",
        CAST(NULL AS DOUBLE) AS "min",
        CAST(NULL AS DOUBLE) AS "p25",
        CAST(NULL AS DOUBLE) AS "p50",
        CAST(NULL AS DOUBLE) AS "p75",
        CAST(NULL AS DOUBLE) AS "max"
      FROM "iris" AS "t0"
    ) AS "t3"
  ) AS "t8"
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      *
    FROM (
      SELECT
        *
      FROM (
        SELECT
          'Sepal.Length' AS "name",
          0 AS "pos",
          'float64' AS "type",
          COUNT("t0"."Sepal.Length" IS NULL) AS "count",
          SUM(CAST("t0"."Sepal.Length" IS NULL AS INT)) AS "nulls",
          COUNT(DISTINCT "t0"."Sepal.Length") AS "unique",
          CAST(NULL AS TEXT) AS "mode",
          AVG("t0"."Sepal.Length") AS "mean",
          STDDEV_SAMP("t0"."Sepal.Length") AS "std",
          MIN("t0"."Sepal.Length") AS "min",
          QUANTILE_CONT("t0"."Sepal.Length", 0.25) AS "p25",
          QUANTILE_CONT("t0"."Sepal.Length", 0.5) AS "p50",
          QUANTILE_CONT("t0"."Sepal.Length", 0.75) AS "p75",
          MAX("t0"."Sepal.Length") AS "max"
        FROM "iris" AS "t0"
      ) AS "t4"
    ) AS "t9"
    UNION ALL
    SELECT
      *
    FROM (
      SELECT
        *
      FROM (
        SELECT
          'Sepal.Width' AS "name",
          1 AS "pos",
          'float64' AS "type",
          COUNT("t0"."Sepal.Width" IS NULL) AS "count",
          SUM(CAST("t0"."Sepal.Width" IS NULL AS INT)) AS "nulls",
          COUNT(DISTINCT "t0"."Sepal.Width") AS "unique",
          CAST(NULL AS TEXT) AS "mode",
          AVG("t0"."Sepal.Width") AS "mean",
          STDDEV_SAMP("t0"."Sepal.Width") AS "std",
          MIN("t0"."Sepal.Width") AS "min",
          QUANTILE_CONT("t0"."Sepal.Width", 0.25) AS "p25",
          QUANTILE_CONT("t0"."Sepal.Width", 0.5) AS "p50",
          QUANTILE_CONT("t0"."Sepal.Width", 0.75) AS "p75",
          MAX("t0"."Sepal.Width") AS "max"
        FROM "iris" AS "t0"
      ) AS "t5"
    ) AS "t10"
  ) AS "t12"
) AS "t13"
```

### Summary statistics

#### Count values

In [53]:
iris.count()
ib.to_sql(iris.count())

```sql
SELECT
  COUNT(*) AS "CountStar(iris)"
FROM "iris" AS "t0"
```

#### Filter

In [64]:
expr = (iris.filter(
    # ib.or_(
        _["Sepal.Width"]>.4,
        _["Species"]=="setosa",
    # )
)#.distinct(on="Species")
)

In [65]:
ib.to_sql(expr)

```sql
SELECT
  *
FROM "iris" AS "t0"
WHERE
  "t0"."Sepal.Width" > 0.4 AND "t0"."Species" = 'setosa'
```

## Read more data

In [66]:
flights = ib.examples.nycflights13_weather.fetch(table_name="flights", backend=conn)

In [67]:
flights.describe()

┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ name     ┃ pos   ┃ type    ┃ count ┃ nulls ┃ unique ┃ mode   ┃ mean        ┃ std      ┃ min     ┃ p25     ┃ p50     ┃ p75     ┃ max     ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ string   │ int16 │ string  │ int64 │ int64 │ int64  │ string │ float64     │ float64  │ float64 │ float64 │ float64 │ float64 │ float64 │
├──────────┼───────┼─────────┼───────┼───────┼────────┼────────┼─────────────┼──────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ pressure │    12 │ string  │ 26115 │     0 │    469 │ NA     │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ visib    │    13 │ float64 │ 26115 │     0 │     20 │ NULL   │    9.255372 │ 2.055052 │     0.0 │    10.0 │    10.0 │    10.0 │    10.0 │
│ origin   │     0 │ string  │ 26115 │     0 │      3 │ JFK    │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ year     │     1 │ int64   │ 26115 │     0 │      1 │ NULL   │ 2013.000000 │ 0.000000 │  2013.0 │  2013.0 │  2013.0 │  2013.0 │  2013.0 │
│ month    │     2 │ int64   │ 26115 │     0 │     12 │ NULL   │    6.503733 │ 3.438328 │     1.0 │     4.0 │     7.0 │     9.0 │    12.0 │
│ day      │     3 │ int64   │ 26115 │     0 │     31 │ NULL   │   15.675321 │ 8.762177 │     1.0 │     8.0 │    16.0 │    23.0 │    31.0 │
│ hour     │     4 │ int64   │ 26115 │     0 │     24 │ NULL   │   11.490791 │ 6.912423 │     0.0 │     6.0 │    11.0 │    17.0 │    23.0 │
│ temp     │     5 │ string  │ 26115 │     0 │    174 │ 37.94  │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ dewp     │     6 │ string  │ 26115 │     0 │    154 │ 28.94  │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ humid    │     7 │ string  │ 26115 │     0 │   2500 │ 100    │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ …        │     … │ …       │     … │     … │      … │ …      │           … │        … │       … │       … │       … │       … │       … │
└──────────┴───────┴─────────┴───────┴───────┴────────┴────────┴─────────────┴──────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

In [ ]:
# ib.to_sql(flights.describe())

In [ ]:
flights_df = flights.execute()

In [73]:
type(flights_df)
flights_df.shape

(10, 15)

In [75]:
flights.count()

┌───────┐
│ 26115 │
└───────┘

In [77]:
flights

┏━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ origin ┃ year  ┃ month ┃ day   ┃ hour  ┃ temp   ┃ dewp   ┃ humid  ┃ wind_dir ┃ wind_speed         ┃ wind_gust ┃ precip  ┃ pressure ┃ visib   ┃ time_hour           ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string │ int64 │ int64 │ int64 │ int64 │ string │ string │ string │ string   │ string             │ string    │ float64 │ string   │ float64 │ timestamp(6)        │
├────────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┼──────────┼────────────────────┼───────────┼─────────┼──────────┼─────────┼─────────────────────┤
│ EWR    │  2013 │     1 │     1 │     1 │ 39.02  │ 26.06  │ 59.37  │ 270      │ 10.357019999999999 │ NA        │     0.0 │ 1012     │    10.0 │ 2013-01-01 06:00:00 │
│ EWR    │  2013 │     1 │     1 │     2 │ 39.02  │ 26.96  │ 61.63  │ 250      │ 8.05546            │ NA        │     0.0 │ 1012.3   │    10.0 │ 2013-01-01 07:00:00 │
│ EWR    │  2013 │     1 │     1 │     3 │ 39.02  │ 28.04  │ 64.43  │ 240      │ 11.5078            │ NA        │     0.0 │ 1012.5   │    10.0 │ 2013-01-01 08:00:00 │
│ EWR    │  2013 │     1 │     1 │     4 │ 39.92  │ 28.04  │ 62.21  │ 250      │ 12.658579999999999 │ NA        │     0.0 │ 1012.2   │    10.0 │ 2013-01-01 09:00:00 │
│ EWR    │  2013 │     1 │     1 │     5 │ 39.02  │ 28.04  │ 64.43  │ 260      │ 12.658579999999999 │ NA        │     0.0 │ 1011.9   │    10.0 │ 2013-01-01 10:00:00 │
│ EWR    │  2013 │     1 │     1 │     6 │ 37.94  │ 28.04  │ 67.21  │ 240      │ 11.5078            │ NA        │     0.0 │ 1012.4   │    10.0 │ 2013-01-01 11:00:00 │
│ EWR    │  2013 │     1 │     1 │     7 │ 39.02  │ 28.04  │ 64.43  │ 240      │ 14.960139999999999 │ NA        │     0.0 │ 1012.2   │    10.0 │ 2013-01-01 12:00:00 │
│ EWR    │  2013 │     1 │     1 │     8 │ 39.92  │ 28.04  │ 62.21  │ 250      │ 10.357019999999999 │ NA        │     0.0 │ 1012.2   │    10.0 │ 2013-01-01 13:00:00 │
│ EWR    │  2013 │     1 │     1 │     9 │ 39.92  │ 28.04  │ 62.21  │ 260      │ 14.960139999999999 │ NA        │     0.0 │ 1012.7   │    10.0 │ 2013-01-01 14:00:00 │
│ EWR    │  2013 │     1 │     1 │    10 │ 41     │ 28.04  │ 59.65  │ 260      │ 13.809359999999998 │ NA        │     0.0 │ 1012.4   │    10.0 │ 2013-01-01 15:00:00 │
│ …      │     … │     … │     … │     … │ …      │ …      │ …      │ …        │ …                  │ …         │       … │ …        │       … │ …                   │
└────────┴───────┴───────┴───────┴───────┴────────┴────────┴────────┴──────────┴────────────────────┴───────────┴─────────┴──────────┴─────────┴─────────────────────┘

In [88]:
(
    flights
    .try_cast({"wind_speed" : "float"})
    .filter(
        _.wind_speed > _.wind_speed.approx_quantile(.99)
    )
    .order_by(
        ib.desc("wind_speed")
    )
)

┏━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ origin ┃ year  ┃ month ┃ day   ┃ hour  ┃ temp   ┃ dewp   ┃ humid  ┃ wind_dir ┃ wind_speed ┃ wind_gust          ┃ precip  ┃ pressure ┃ visib   ┃ time_hour           ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string │ int64 │ int64 │ int64 │ int64 │ string │ string │ string │ string   │ float64    │ string             │ float64 │ string   │ float64 │ timestamp(6)        │
├────────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┼──────────┼────────────┼────────────────────┼─────────┼──────────┼─────────┼─────────────────────┤
│ EWR    │  2013 │     2 │    12 │     3 │ 39.02  │ 26.96  │ 61.63  │ 260      │ 1048.36058 │ NA                 │    0.00 │ 1008.3   │    10.0 │ 2013-02-12 08:00:00 │
│ JFK    │  2013 │     1 │    31 │     4 │ 53.6   │ 53.06  │ 100    │ 200      │   42.57886 │ 52.93588           │    0.20 │ NA       │     2.0 │ 2013-01-31 09:00:00 │
│ EWR    │  2013 │     1 │    31 │     6 │ 57.2   │ 53.6   │ 87.74  │ 270      │   42.57886 │ 55.23743999999999  │    0.03 │ NA       │    10.0 │ 2013-01-31 11:00:00 │
│ EWR    │  2013 │     1 │    31 │     4 │ 60.8   │ 59     │ 93.79  │ 230      │   40.27730 │ 58.68978           │    0.30 │ NA       │     6.0 │ 2013-01-31 09:00:00 │
│ LGA    │  2013 │     1 │    31 │     4 │ 59     │ 55.4   │ 93.69  │ 230      │   40.27730 │ 55.23743999999999  │    0.41 │ NA       │     8.0 │ 2013-01-31 09:00:00 │
│ EWR    │  2013 │     1 │    31 │     8 │ 46.04  │ 30.02  │ 53.33  │ 270      │   39.12652 │ 48.33275999999999  │    0.00 │ 992.3    │    10.0 │ 2013-01-31 13:00:00 │
│ JFK    │  2013 │     3 │     6 │    14 │ 41     │ 28.94  │ 61.89  │ 50       │   37.97574 │ 46.0312            │    0.00 │ NA       │    10.0 │ 2013-03-06 19:00:00 │
│ JFK    │  2013 │    11 │    24 │    10 │ 28.04  │ -0.04  │ 29.16  │ 310      │   36.82496 │ 47.181979999999996 │    0.00 │ 1021.5   │    10.0 │ 2013-11-24 15:00:00 │
│ JFK    │  2013 │     1 │    31 │     7 │ 51.8   │ 46.4   │ 81.74  │ 270      │   36.82496 │ 58.68978           │    0.00 │ NA       │    10.0 │ 2013-01-31 12:00:00 │
│ JFK    │  2013 │     1 │    31 │     3 │ 53.06  │ 51.98  │ 100    │ 180      │   36.82496 │ 48.33275999999999  │    0.07 │ NA       │     1.0 │ 2013-01-31 08:00:00 │
│ …      │     … │     … │     … │     … │ …      │ …      │ …      │ …        │          … │ …                  │       … │ …        │       … │ …                   │
└────────┴───────┴───────┴───────┴───────┴────────┴────────┴────────┴──────────┴────────────┴────────────────────┴─────────┴──────────┴─────────┴─────────────────────┘

## Raw DuckDB + IBIS

### Selecting

In [23]:
# SQL
select_sql = conn.sql("""
    SELECT "Petal.Width", Species FROM iris_raw;
""")
select_sql

┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Petal.Width ┃ Species ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ float64     │ string  │
├─────────────┼─────────┤
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.4 │ setosa  │
│         0.3 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.1 │ setosa  │
│           … │ …       │
└─────────────┴─────────┘

In [39]:
# ibis
select_expr = iris.select("Petal.Width", "Species")
select_expr

┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Petal.Width ┃ Species ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ float64     │ string  │
├─────────────┼─────────┤
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.4 │ setosa  │
│         0.3 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.1 │ setosa  │
│           … │ …       │
└─────────────┴─────────┘

In [40]:
ib.to_sql(select_expr)

```sql
SELECT
  "t0"."Petal.Width",
  "t0"."Species"
FROM "iris_raw" AS "t0"
```

### Filtering

In [41]:
# SQL
filter_sql = conn.sql("""
    SELECT * from iris_raw where "Petal.Width" > .3;
""")
filter_sql

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sepal.Length ┃ Sepal.Width ┃ Petal.Length ┃ Petal.Width ┃ Species    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64      │ float64     │ float64      │ float64     │ string     │
├──────────────┼─────────────┼──────────────┼─────────────┼────────────┤
│          5.4 │         3.9 │          1.7 │         0.4 │ setosa     │
│          5.7 │         4.4 │          1.5 │         0.4 │ setosa     │
│          5.4 │         3.9 │          1.3 │         0.4 │ setosa     │
│          5.1 │         3.7 │          1.5 │         0.4 │ setosa     │
│          5.1 │         3.3 │          1.7 │         0.5 │ setosa     │
│          5.0 │         3.4 │          1.6 │         0.4 │ setosa     │
│          5.4 │         3.4 │          1.5 │         0.4 │ setosa     │
│          5.0 │         3.5 │          1.6 │         0.6 │ setosa     │
│          5.1 │         3.8 │          1.9 │         0.4 │ setosa     │
│          7.0 │         3.2 │          4.7 │         1.4 │ versicolor │
│            … │           … │            … │           … │ …          │
└──────────────┴─────────────┴──────────────┴─────────────┴────────────┘

In [46]:
# ibis
iris.filter(_["Petal.Width"] > .3)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sepal.Length ┃ Sepal.Width ┃ Petal.Length ┃ Petal.Width ┃ Species    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64      │ float64     │ float64      │ float64     │ string     │
├──────────────┼─────────────┼──────────────┼─────────────┼────────────┤
│          5.4 │         3.9 │          1.7 │         0.4 │ setosa     │
│          5.7 │         4.4 │          1.5 │         0.4 │ setosa     │
│          5.4 │         3.9 │          1.3 │         0.4 │ setosa     │
│          5.1 │         3.7 │          1.5 │         0.4 │ setosa     │
│          5.1 │         3.3 │          1.7 │         0.5 │ setosa     │
│          5.0 │         3.4 │          1.6 │         0.4 │ setosa     │
│          5.4 │         3.4 │          1.5 │         0.4 │ setosa     │
│          5.0 │         3.5 │          1.6 │         0.6 │ setosa     │
│          5.1 │         3.8 │          1.9 │         0.4 │ setosa     │
│          7.0 │         3.2 │          4.7 │         1.4 │ versicolor │
│            … │           … │            … │           … │ …          │
└──────────────┴─────────────┴──────────────┴─────────────┴────────────┘

## Ecosystem

Extensions and applications
- spatial
- OSM data reading
- String matching
- scalenav

## Examples

### Example 1 : US BEA economic output data

### Example 2: OSM